In [19]:
import numpy as np
import pandas as pd
import os
from difflib import SequenceMatcher
import xlwt
from pandas import ExcelWriter
from sklearn.metrics.pairwise import cosine_similarity 

import plotly.graph_objects as go

The below commented code is for merging the files. As the files were already merged I chose to comment it. 

In [2]:
#cwd = os.getcwd()
#path = os.walk(".")

In [10]:
'''
shooting = []
shooting = pd.DataFrame()

for directories in os.listdir(cwd):
    #print(directories)
    if directories.endswith('.ipynb'):
        continue
    if directories.endswith('.csv'):
        continue
        
    else:
        
        for file in os.listdir(directories):
           # print(file)
            if file.startswith('shooting.xls') :
                
              #  os.rename(os.path.join(directories,file),os.path.join(directories,'aa_std_stats.xls'))    
                data = pd.read_html(os.path.join(directories,file),encoding = 'utf-8')
                #print(data[0])
                data = pd.DataFrame(data[0])
                data.columns = data.columns.droplevel()
                data['club'] = directories
                #print(data)
                shooting = shooting.append(data,ignore_index = True)
               # print("found")
               
'''               

In [11]:
#shooting.to_csv('20-21_shooting.csv')

In [5]:
shooting = pd.read_csv('19-20_shooting.csv')

In [7]:
shooting.isnull().sum()

Unnamed: 0      0
Player          0
Nation          2
Pos             2
Age             1
90s             0
Gls             0
Sh             12
SoT            12
SoT%          581
Sh/90          12
SoT/90         12
G/Sh          581
G/SoT         944
Dist          681
FK            175
PK              0
PKatt           0
xG            175
npxG          175
npxG/Sh       683
G-xG          175
np:G-xG       175
Matches         2
club            0
dtype: int64

In [10]:


df_final = shooting.drop(columns=['Matches','club','Nation','Age','Unnamed: 0','SoT%','G/Sh','G/SoT','Dist','npxG/Sh'])


df_final.dropna(inplace=True)

In [12]:
df_final.columns = ['Player','Position','90s','Goals',
                   'Shots','Shots on Target','Total shots per 90',
                   'Shot on Target per 90min','Free Kicks','Penalty Kicks',
                   'Penalty kicks attempted','Expected Goals','Non penalty expected goals','Goals-Expected goals',
                   'Non Penalty goals - Non penalty expected goals']

In [14]:
to_be_scaled = df_final[['90s','Goals',
                   'Shots','Shots on Target','Total shots per 90',
                   'Shot on Target per 90min','Free Kicks','Penalty Kicks',
                   'Penalty kicks attempted','Expected Goals','Non penalty expected goals','Goals-Expected goals',
                   'Non Penalty goals - Non penalty expected goals']]

# copy the data
df_z_scaled = to_be_scaled.copy()
  
# apply normalization techniques
for column in df_z_scaled.columns:
    df_z_scaled[column] = (df_z_scaled[column] -
                           df_z_scaled[column].mean()) / df_z_scaled[column].std()    
  
# view normalized data   
display(df_z_scaled)

,90s,Goals,Shots,Shots on Target,Total shots per 90,Shot on Target per 90min,Free Kicks,Penalty Kicks,Penalty kicks attempted,Expected Goals,Non penalty expected goals,Goals-Expected goals,Non Penalty goals - Non penalty expected goals
0,1.504692,-0.522112,-0.754490,-0.653082,-0.526583,-0.228316,-0.291099,-0.228405,-0.244314,-0.564914,-0.600001,0.001651,0.011324
1,1.280116,1.546631,2.789183,2.127054,0.630795,0.233335,1.076102,1.741900,3.149854,2.986344,2.476626,-2.671190,-1.796283
2,1.211016,-0.263519,0.164240,0.014151,-0.217659,-0.115718,1.531835,-0.228405,-0.244314,0.037532,0.121677,-0.750085,-0.763364
3,0.986440,-0.522112,-0.491996,-0.430671,-0.430860,-0.188907,-0.291099,-0.228405,-0.244314,-0.247837,-0.220171,-0.833612,-0.849441
4,0.891427,-0.263519,-0.229502,-0.208260,-0.326435,-0.143868,-0.291099,-0.228405,-0.244314,-0.342960,-0.334120,0.252230,0.269554
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2954,-1.095207,-0.522112,-0.710741,-0.541877,-0.378647,-0.036900,-0.291099,-0.228405,-0.244314,-0.564914,-0.600001,0.001651,0.011324
2955,-1.121120,-0.522112,-0.754490,-0.653082,-0.526583,-0.228316,-0.291099,-0.228405,-0.244314,-0.564914,-0.600001,0.001651,0.011324
2956,-1.164307,-0.522112,-0.579494,-0.653082,0.313169,-0.228316,-0.291099,-0.228405,-0.244314,-0.374668,-0.372103,-0.499506,-0.505135
2957,-1.242045,-0.522112,-0.623243,-0.653082,0.591636,-0.228316,-0.291099,-0.228405,-0.244314,-0.501498,-0.524035,-0.165401,-0.160829


In [15]:

df_z_scaled.insert(0, "Player", df_final['Player'])
df_z_scaled.insert(1, "Position", df_final['Position'])

df_final = df_z_scaled

df_final

,Player,Position,90s,Goals,Shots,Shots on Target,Total shots per 90,Shot on Target per 90min,Free Kicks,Penalty Kicks,Penalty kicks attempted,Expected Goals,Non penalty expected goals,Goals-Expected goals,Non Penalty goals - Non penalty expected goals
0,Fernando Pacheco,GK,1.504692,-0.522112,-0.754490,-0.653082,-0.526583,-0.228316,-0.291099,-0.228405,-0.244314,-0.564914,-0.600001,0.001651,0.011324
1,Joselu,FW,1.280116,1.546631,2.789183,2.127054,0.630795,0.233335,1.076102,1.741900,3.149854,2.986344,2.476626,-2.671190,-1.796283
2,Florian Lejeune,DF,1.211016,-0.263519,0.164240,0.014151,-0.217659,-0.115718,1.531835,-0.228405,-0.244314,0.037532,0.121677,-0.750085,-0.763364
3,Víctor Laguardia,DF,0.986440,-0.522112,-0.491996,-0.430671,-0.430860,-0.188907,-0.291099,-0.228405,-0.244314,-0.247837,-0.220171,-0.833612,-0.849441
4,Rodrigo Battaglia,MF,0.891427,-0.263519,-0.229502,-0.208260,-0.326435,-0.143868,-0.291099,-0.228405,-0.244314,-0.342960,-0.334120,0.252230,0.269554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2954,Rúben Vinagre,DF,-1.095207,-0.522112,-0.710741,-0.541877,-0.378647,-0.036900,-0.291099,-0.228405,-0.244314,-0.564914,-0.600001,0.001651,0.011324
2955,Morgan Gibbs-White,"FW,MF",-1.121120,-0.522112,-0.754490,-0.653082,-0.526583,-0.228316,-0.291099,-0.228405,-0.244314,-0.564914,-0.600001,0.001651,0.011324
2956,Owen Otasowie,"MF,FW",-1.164307,-0.522112,-0.579494,-0.653082,0.313169,-0.228316,-0.291099,-0.228405,-0.244314,-0.374668,-0.372103,-0.499506,-0.505135
2957,Patrick Cutrone,"FW,MF",-1.242045,-0.522112,-0.623243,-0.653082,0.591636,-0.228316,-0.291099,-0.228405,-0.244314,-0.501498,-0.524035,-0.165401,-0.160829


In [16]:
def get_similar_players(player_name):
    
    player_index = [list(df_final['Player']).index(x) for x in list(df_final['Player']) if player_name in x]
    player_index = int(player_index[0])
    print(player_index)
    df1 = df_final.iloc[:,5:]
    cos = cosine_similarity(df1,df1)
    player_cos = sorted(list(cos[player_index]))[-7:-1]
    indexes = [list(cos[player_index]).index(x) for x in player_cos]
    indexes.append(player_index)
    return player_cos, df_final.iloc[indexes]

In [23]:
value = get_similar_players('Romelu Lukaku')

1111


In [24]:
categories = ['Position','90s','Goals',
                   'Shots','Shots on Target','Total shots per 90',
                   'Shot on Target per 90min','Free Kicks','Penalty Kicks',
                   'Penalty kicks attempted','Expected Goals','Non penalty expected goals','Goals-Expected goals',
                   'Non Penalty goals - Non penalty expected goals']



fig = go.Figure()
r = []
for i in range(7):
    
    data = np.array(value[1].iloc[i])
    
    fig.add_trace(go.Scatterpolar(
      r=data,     
      theta=categories,
      fill='toself',
      name=value[1]['Player'].iloc[i]
    ))


fig.update_layout(
  
  showlegend=True
)

fig.show()